In [10]:
import torch
from torch import nn
from torch.nn import functional as F

## 自定义块

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))

In [12]:
x = torch.randn(2, 20)
net = MLP()
net(x)

tensor([[ 0.3716,  0.0121, -0.0073, -0.3525, -0.1089, -0.0733,  0.1065,  0.3618,
          0.1939, -0.1517],
        [ 0.5724, -0.0104, -0.2306, -0.3068, -0.2287, -0.1087, -0.1530,  0.0443,
         -0.0157, -0.1221]], grad_fn=<AddmmBackward0>)

## 顺序块

In [21]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    def forward(self, x):
        for block in self._modules.values():
            x = block(x)
        return x

In [22]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
net(x)

tensor([[-0.3801,  0.2461,  0.1130, -0.3972, -0.0480,  0.1362, -0.2885, -0.3651,
          0.2276, -0.2831],
        [-0.1988,  0.0601,  0.1841, -0.3181, -0.2749,  0.1416, -0.2165, -0.1323,
          0.3200, -0.1270]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super(FixedHiddenMLP, self).__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, x):
        x = self.linear(x)
        x = F.relu(torch.mm(x, self.rand_weight)+1)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()

In [7]:
net = FixedHiddenMLP()
net(x)

tensor(0.0237, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super(NestMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
    def forward(self, x):
        return self.linear(self.net(x))

In [9]:
chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
chimera(x)

tensor(0.1433, grad_fn=<SumBackward0>)

## 效率

## Exercises

1. 如果将MySequential中存储块的方式更改为Python列表，会出现什么样的问题？

2. 实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的串联输出。这也被称为平行块

3. 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络